# Blcok Recognition

## Import Libraries

In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy
import pickle

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('./test_imgs/dep.p', 'rb') as dep:
    d = pickle.load(dep, encoding="16UC1")
    
d

array([[403, 404, 404, ..., 797, 797, 797],
       [404, 404, 404, ..., 796, 796, 797],
       [404, 404, 404, ..., 796, 796, 796],
       ...,
       [144, 144, 144, ..., 137, 137, 137],
       [144, 144, 144, ..., 137, 137, 137],
       [144, 144, 144, ..., 137, 137, 137]], dtype=uint16)

In [3]:
with open('./test_imgs/rgb.p', 'rb') as rgb:
    r = pickle.load(rgb)
    
r

array([[[ 88, 122, 111],
        [103, 137, 126],
        [111, 139, 126],
        ...,
        [175, 175, 159],
        [177, 174, 159],
        [177, 174, 159]],

       [[107, 144, 128],
        [116, 153, 137],
        [121, 150, 134],
        ...,
        [166, 174, 157],
        [166, 174, 157],
        [165, 173, 156]],

       [[110, 148, 130],
        [113, 151, 133],
        [118, 148, 129],
        ...,
        [165, 173, 156],
        [165, 173, 156],
        [165, 173, 156]],

       ...,

       [[182, 210, 187],
        [184, 212, 189],
        [187, 213, 190],
        ...,
        [246, 213, 194],
        [246, 213, 194],
        [246, 213, 194]],

       [[181, 211, 186],
        [183, 213, 188],
        [186, 212, 189],
        ...,
        [245, 212, 193],
        [247, 211, 193],
        [247, 211, 193]],

       [[178, 210, 185],
        [180, 212, 187],
        [184, 212, 189],
        ...,
        [246, 210, 192],
        [248, 210, 192],
        [248, 210, 192]]

In [4]:
d.min()

0

In [5]:
d1 = np.asarray(d, dtype = np.float32)

In [6]:
d2 = d1/915
d2 = np.array(d2, dtype=np.float32)
d2.shape

(1536, 2048)

In [7]:
d3 = np.array(d2 * 255, dtype='uint8')

In [8]:
# cv2.imshow('d2', d3)
# cv2.imwrite('../imgs/depth.png', d3)
# # cv2.imshow('r', r)
# # cv2.imwrite('../imgs/color.png', r)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## Read Test Images

In [9]:
# img_color = cv2.imread('test_imgs/jenga_tower_color.png')
# img_depth = cv2.imread('test_imgs/jenga_tower_depth.png', 0)
# img_color = cv2.imread('/home/shs/Desktop/rgb1.png')
# img_depth = cv2.imread('/home/shs/Desktop/dep.png', 0)
img_color = r
img_depth = d
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (1536, 2048, 3)
depth shape:  (1536, 2048)


## Extract Blocks' Masks By Colors

In [10]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [11]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [12]:
# RED
lower_red1 = np.array([0, 130, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,130,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 70, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,70,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (70-20, 50, 50)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 60, 60)
upper_violet = (130+20, 255, 255)

In [13]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2
        
        # Denoise by Erosion and Dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    # if color == 'red':
    #     cv2.imshow('src_bin', src_bin)
    #     cv2.imwrite('../imgs/red_mask.png', src_bin)
    #     cv2.imshow(f'{color}_filtered', each_color_filtered)
    #     cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 700:
            continue
        
        # if color == 'red' and i==2:
        #     cv2.imshow('blk clr', block_color)
        #     cv2.imwrite('../imgs/red_block2_color.png', block_color)
        #     cv2.imshow('blk msk', block_mask)
        #     cv2.imwrite('../imgs/red_block2_mask.png', block_mask)
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()
            
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [14]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [15]:
cv2.imshow('tower mask', tower_mask)
cv2.imwrite('../imgs/tower_mask.png', tower_mask)
cv2.imshow('tower color', tower_color)
cv2.imwrite('../imgs/tower_color.png', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b), "Blocks")

green
6 Blocks
pink
6 Blocks
yellow
6 Blocks
blue
6 Blocks
violet
6 Blocks
red
6 Blocks


## Get PointCloud from RGB Image + Depth Image

In [17]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[968.813, 0, 1023.83],
                              [0, 968.635, 775.975],
                              [0, 0, 1]]


# extrinsic = np.array([[0.999987, 0.00512943, 0.000573447, -32.0559],
#                      [-0.00516119, 0.994725, 0.102443, -1.91788],
#                      [-4.49478e-05, -0.102445, 0.994739, 3.99916],
#                      [0, 0, 0, 1]])
# extrinsic

In [18]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50, origin=[0, 0, 0])
mesh_frame_small = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=[0, 0, 0])

In [19]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, depth_scale=1, depth_trunc=3000.0, convert_rgb_to_intensity=False)
    # print(np.asarray(rgbd_image.depth).max())
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [20]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [21]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)

In [22]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 269250 points.
[[-17.71595757 -77.9030543  215.        ]
 [-17.57540413 -78.26539409 216.        ]
 [-17.35245089 -78.26539409 216.        ]
 ...
 [-17.76493503 107.56967279 230.        ]
 [-17.52753111 107.56967279 230.        ]
 [-17.2901272  107.56967279 230.        ]]


In [23]:
points = np.asarray(tower_pcd.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -191.02763897676851 52.882816394907984
Y min/max: -316.3226344288613 110.35418398054993
Z min/max: 215.0 889.0


In [24]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    # if color != 'blue':
    #     continue
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(1024, 25)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        # if color=='blue':
        #     o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [25]:
len(blocks_pcd_by_color)

6

## Tower ICP

In [26]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
# print(len(pcd_combined.points))

PointCloud with 268571 points.


In [27]:
points = np.asarray(pcd_combined.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -61.22128831879838 52.882816394907984
Y min/max: -81.87782291575259 110.35418398054993
Z min/max: 215.0 307.0


In [28]:

# o3d.visualization.draw_geometries([pcd_combined, mesh_frame_small])

o3d.visualization.draw_geometries([pcd_combined])

In [29]:
mesh_tower = o3d.io.read_triangle_mesh("mesh/jenga_tower_side_xy.stl")
print(mesh_tower)

mesh_tower.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh])

pcd_target = mesh_tower.sample_points_uniformly(number_of_points=len(pcd_combined.points))
print(pcd_target)
o3d.visualization.draw_geometries([pcd_target])

TriangleMesh with 12 points and 4 triangles.
PointCloud with 268571 points.


In [30]:
np.linalg.inv(np.asarray([[0, 0, -1, 0],
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, 0, 1]]))

array([[ 0.,  1.,  0.,  0.],
       [-0., -0., -1., -0.],
       [-1., -0., -0., -0.],
       [ 0.,  0.,  0.,  1.]])

In [31]:
initial_transform = np.asarray([[0, 0, -1, 0],
                                [1, 0, 0, 0],
                                [0, -1, 0, 0],
                                [0, 0, 0, 1]])

In [32]:
def prepare_icp(source, target):
    source_tmp = copy.deepcopy(source)
    target_tmp = copy.deepcopy(target)
    
    # make the point cloud into right position
    # source_tmp.transform(initial_transform)
    
    # resize the target pointcloud to make two pointclouds into same scale
    # resize =(np.array(target_tmp.points)[:,2].max() - np.array(target_tmp.points)[:,2].min()) / (np.array(source_tmp.points)[:,2].max() - np.array(source_tmp.points)[:,2].min())
    
    # move the source pcd to do icp
    move = np.array(target_tmp.get_center() - source_tmp.get_center())
    
    # a = o3d.cpu.pybind.utility.Vector3dVector(np.array(target_tmp.points))
    # b = o3d.cpu.pybind.utility.Vector3dVector(np.array(source_tmp.points))
    
    # target_tmp.points = a
    # source_tmp.points = b
    
    o3d.visualization.draw_geometries([source_tmp, target_tmp, mesh_frame])
    
    return source_tmp, target_tmp, move

In [33]:
source, target,  move = prepare_icp(pcd_combined, pcd_target)

In [34]:
move

array([  21.38104116,    6.53476951, -163.3909975 ])

In [35]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp, mesh_frame])

In [36]:
threshold = 10
trans_init = np.asarray([[0, 0, -1, 256.1241073],
                         [1, 0, 0, 21.20056151],
                         [0, -1, 0, 101.30002853],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init)

In [37]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=2.049253e-01, inlier_rmse=5.852899e+00, and correspondence_set size of 55037
Access transformation to get result.


In [38]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=9.985069e-01, inlier_rmse=2.497088e+00, and correspondence_set size of 268170
Access transformation to get result.
Transformation is:
[[-8.61747931e-01  1.90546212e-02 -5.06978722e-01  1.39680360e+02]
 [ 5.06785363e-01 -1.42432957e-02 -8.61954595e-01  2.45085516e+02]
 [-2.36452662e-02 -9.99716985e-01  2.61752747e-03  1.01377365e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [39]:
draw_registration_result(source, target, reg_p2p.transformation)

In [40]:
source_temp = copy.deepcopy(source)

source_temp.transform(reg_p2p.transformation)


PointCloud with 268571 points.

In [41]:
(np.array(source_temp.points))[:,2].min()

-7.805627756627189

## Blocks Transformation

In [43]:
trans = reg_p2p.transformation

In [44]:
trans

array([[-8.61747931e-01,  1.90546212e-02, -5.06978722e-01,
         1.39680360e+02],
       [ 5.06785363e-01, -1.42432957e-02, -8.61954595e-01,
         2.45085516e+02],
       [-2.36452662e-02, -9.99716985e-01,  2.61752747e-03,
         1.01377365e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [49]:
def transform_blocks(pcd, icp_transform):
    pcd_temp = copy.deepcopy(pcd)
    aa = o3d.cpu.pybind.utility.Vector3dVector(np.array(pcd_temp.points))
    pcd_temp.points = aa
    pcd_temp.transform(icp_transform)
    
    return pcd_temp

In [50]:
# WHAT IS THE TARGET BLOCK?
target_block_color = 'red'
target_block_label = 2

In [51]:
for col, pcds in zip(colors, blocks_pcd_by_color):
    # if col != target_block_color:
    #     continue
    print(col)
    
    for idx, pcd in enumerate(pcds):
        if idx != target_block_label:
            continue
        
        # print("--------------------------------")
        print(idx)
        pcd_new = transform_blocks(pcd, trans)
        
        # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
        box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
        print("BOX EXTENT : ", box_extent)
        
        center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
        # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
        print("BOX CENTER COORDINATE : ", center_coordinate)
        
        # print("BOX MAX X,Y and MEAN Z Coordinate")
        x_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,0].mean()
        y_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,1].mean()
        z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
        
        if box_extent[1] > 70:
            print("PULL DIRECTION : X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean #- 25/2
            cen_y = y_mean #- 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 100
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[0] > 70:
            print("PULL DIRECTION : Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 100
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[0]) < 10 and box_extent [1] < 15:
            print("PUSH DIRECTION : Y or -Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 25/2
            cen_y = y_mean - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 150
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[1]) < 10 and box_extent [0] < 15:
            print("PUSH DIRECTION : X or -X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 150
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[1] < 15:
            print("PULL DIRECTION : -X")
            vector = np.array([1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 25/2
            cen_y = y_mean - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x - 100
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[0] < 15:
            print("PULL DIRECTION : -Y")
            vector = np.array([0, 1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y - 100
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        else:
            print("NOTHING")
            
        mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
        ptc1 = o3d.geometry.PointCloud()
        ptc1.points = pt1
        ptc1.paint_uniform_color([1, 0, 0])
        ptc2 = o3d.geometry.PointCloud()
        ptc2.points = pt2
        ptc2.paint_uniform_color([0, 0, 1])
        line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
        o3d.visualization.draw_geometries([pcd_new, source_temp, mesh_frame, ptc1, ptc2, line])

green
2
BOX EXTENT :  [38.49367106 84.14673802 16.79570164]
BOX CENTER COORDINATE :  [28.51200225 -4.40687754 99.83280373]
PULL DIRECTION : X
-----BLOCK CENTER COORDINATE-----
X : 28.512002249426864
Y : -4.40687753819293
Z : 99.83280372626282
-----TARGET COORDINATE-----
X : 128.51200224942687
Y : -4.40687753819293
Z : 99.83280372626282
----------------------------------
pink
2
BOX EXTENT :  [ 9.03295567 17.44428824 15.55907434]
BOX CENTER COORDINATE :  [ 39.96560922 -30.99186588 117.4615336 ]
PULL DIRECTION : -Y
-----BLOCK CENTER COORDINATE-----
X : 2.4656092174260067
Y : -30.99186588360061
Z : 117.46153360027262
-----TARGET COORDINATE-----
X : 2.4656092174260067
Y : -130.9918658836006
Z : 117.46153360027262
----------------------------------
yellow
2
BOX EXTENT :  [27.29089313  7.66915903 14.97085825]
BOX CENTER COORDINATE :  [-28.88857695  39.04462238  98.63514137]
PULL DIRECTION : -X
-----BLOCK CENTER COORDINATE-----
X : -28.8885769532628
Y : 1.5446223832913688
Z : 98.63514136537493

In [52]:
mesh_block = o3d.io.read_triangle_mesh("mesh/single_block_x.stl")
print(mesh_block)

mesh_block.compute_vertex_normals()

TriangleMesh with 96 points and 32 triangles.


TriangleMesh with 96 points and 32 triangles.

In [53]:
# WHAT IS THE TARGET BLOCK?
target_block_color = 'red'
target_block_label = 2

In [56]:
# ICP 콜백 함수 정의
def registration_callback(src, tgt, current_transformation):
    # X, Y 축에 대한 회전을 고정하기 위해 해당 부분을 수정합니다.
    # Z 축에 대한 변환을 찾기 위해 ICP 알고리즘을 사용합니다.
    current_transformation[:2, :2] = np.identity(2)  # X, Y 축 회전 고정

    return current_transformation

In [66]:
for col, pcds in zip(colors, blocks_pcd_by_color):
    if col != target_block_color:
        continue
    print(col)
    
    for idx, pcd in enumerate(pcds):
        if idx != target_block_label:
            continue
        
        # print("--------------------------------")
        print(idx)
        pcd_new = transform_blocks(pcd, trans)

        z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()

        print(z_mean)
        
        # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
        box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
        print("BOX EXTENT : ", box_extent)
        
        center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
        # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
        print("BOX CENTER COORDINATE : ", center_coordinate)
        
        
        block_source = mesh_block.sample_points_uniformly(number_of_points=len(pcd_new.points))
        print(block_source)
        o3d.visualization.draw_geometries([block_source])
        block_source, target_pcd, move = prepare_icp(source=block_source, target=pcd_new)
        
        threshold = 10
        trans_init = np.asarray([[1, 0, 0, move[0]],
                                [0, 1, 0, move[1]],
                                [0, 0, 1, move[2]],
                                [0, 0, 0, 1]])
        

        draw_registration_result(block_source, target_pcd, trans_init)
    
        print("Initial alignment")
        evaluation = o3d.pipelines.registration.evaluate_registration(
            block_source, target_pcd, threshold, trans_init)
        print(evaluation)
        
        reg_p2p = o3d.pipelines.registration.registration_icp(
            block_source, target_pcd, threshold, trans_init,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
        print(reg_p2p)
        print("Transformation is:")
        print(reg_p2p.transformation)
    
        draw_registration_result(block_source, target_pcd, reg_p2p.transformation)
        
        print("NEW Transformation is:")
        r11 = reg_p2p.transformation[0,0]
        r21 = reg_p2p.transformation[1,0]

        theta_z = np.arctan2(r21, r11)
        trans_z = copy.deepcopy(reg_p2p.transformation)
        trans_z[0,0] = np.cos(theta_z)
        trans_z[0,1] = -np.sin(theta_z)
        trans_z[0,2] = 0
        trans_z[1,0] = np.sin(theta_z)
        trans_z[1,1] = np.cos(theta_z)
        trans_z[1,2] = 0
        trans_z[2,0] = 0
        trans_z[2,1] = 0
        trans_z[2,2] = 1
        
        
        print(trans_z)
    
        draw_registration_result(block_source, target_pcd, trans_z)

        pcd_new1 = transform_blocks(block_source, trans_z)
        # pcd_new1 = 

        
        
        # print("BOX MAX X,Y and MEAN Z Coordinate")
        x_mean = np.array(pcd_new1.get_axis_aligned_bounding_box().get_box_points())[:,0].mean()
        y_mean = np.array(pcd_new1.get_axis_aligned_bounding_box().get_box_points())[:,1].mean()
        z_mean = np.array(pcd_new1.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
        print(x_mean, y_mean, z_mean)
        
        # if box_extent[1] > 70:
        #     print("PULL DIRECTION : X")
        #     vector = np.array([-1, 0, 0])
        #     print("-----BLOCK CENTER COORDINATE-----")
        #     cen_x = x_mean #- 25/2
        #     cen_y = y_mean #- 75/2
        #     cen_z = z_mean
        #     print("X :", cen_x)
        #     print("Y :", cen_y)
        #     print("Z :", cen_z)
        #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
        #     print("-----TARGET COORDINATE-----")
        #     target_x = cen_x + 100
        #     target_y = cen_y
        #     target_z = cen_z
        #     print("X :", target_x)
        #     print("Y :", target_y)
        #     print("Z :", target_z)
        #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
        #     print("----------------------------------")
            
        # elif box_extent[0] > 70:
        #     print("PULL DIRECTION : Y")
        #     vector = np.array([0, -1, 0])
        #     print("-----BLOCK CENTER COORDINATE-----")
        #     cen_x = x_mean# - 75/2
        #     cen_y = y_mean# - 25/2
        #     cen_z = z_mean
        #     print("X :", cen_x)
        #     print("Y :", cen_y)
        #     print("Z :", cen_z)
        #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
        #     print("-----TARGET COORDINATE-----")
        #     target_x = cen_x
        #     target_y = cen_y + 100
        #     target_z = cen_z
        #     print("X :", target_x)
        #     print("Y :", target_y)
        #     print("Z :", target_z)
        #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
        #     print("----------------------------------")
            
        # elif abs(center_coordinate[0]) < 10 and box_extent [1] < 20:
        #     print("PUSH DIRECTION : Y or -Y")
        #     vector = np.array([0, -1, 0])
        #     print("-----BLOCK CENTER COORDINATE-----")
        #     cen_x = x_mean# - 25/2
        #     cen_y = y_mean - 75/2
        #     cen_z = z_mean
        #     print("X :", cen_x)
        #     print("Y :", cen_y)
        #     print("Z :", cen_z)
        #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
        #     print("-----TARGET COORDINATE-----")
        #     target_x = cen_x
        #     target_y = cen_y + 150
        #     target_z = cen_z
        #     print("X :", target_x)
        #     print("Y :", target_y)
        #     print("Z :", target_z)
        #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
        #     print("----------------------------------")
            
        # elif abs(center_coordinate[1]) < 10 and box_extent [0] < 20:
        #     print("PUSH DIRECTION : X or -X")
        #     vector = np.array([-1, 0, 0])
        #     print("-----BLOCK CENTER COORDINATE-----")
        #     cen_x = x_mean - 75/2
        #     cen_y = y_mean# - 25/2
        #     cen_z = z_mean
        #     print("X :", cen_x)
        #     print("Y :", cen_y)
        #     print("Z :", cen_z)
        #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
        #     print("-----TARGET COORDINATE-----")
        #     target_x = cen_x + 150
        #     target_y = cen_y
        #     target_z = cen_z
        #     print("X :", target_x)
        #     print("Y :", target_y)
        #     print("Z :", target_z)
        #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
        #     print("----------------------------------")
            
        # elif box_extent[1] < 20:
        #     print("PULL DIRECTION : -X")
        #     vector = np.array([1, 0, 0])
        #     print("-----BLOCK CENTER COORDINATE-----")
        #     cen_x = x_mean# - 25/2
        #     cen_y = y_mean - 75/2
        #     cen_z = z_mean
        #     print("X :", cen_x)
        #     print("Y :", cen_y)
        #     print("Z :", cen_z)
        #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
        #     print("-----TARGET COORDINATE-----")
        #     target_x = cen_x - 100
        #     target_y = cen_y
        #     target_z = cen_z
        #     print("X :", target_x)
        #     print("Y :", target_y)
        #     print("Z :", target_z)
        #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
        #     print("----------------------------------")
            
        # elif box_extent[0] < 20:
        #     print("PULL DIRECTION : -Y")
        #     vector = np.array([0, 1, 0])
        #     print("-----BLOCK CENTER COORDINATE-----")
        #     cen_x = x_mean - 75/2
        #     cen_y = y_mean# - 25/2
        #     cen_z = z_mean
        #     print("X :", cen_x)
        #     print("Y :", cen_y)
        #     print("Z :", cen_z)
        #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
        #     print("-----TARGET COORDINATE-----")
        #     target_x = cen_x
        #     target_y = cen_y - 100
        #     target_z = cen_z
        #     print("X :", target_x)
        #     print("Y :", target_y)
        #     print("Z :", target_z)
        #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
        #     print("----------------------------------")
            
        # else:
        #     print("NOTHING")
            
        # mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
        # ptc1 = o3d.geometry.PointCloud()
        # ptc1.points = pt1
        # ptc1.paint_uniform_color([1, 0, 0])
        # ptc2 = o3d.geometry.PointCloud()
        # ptc2.points = pt2
        # ptc2.paint_uniform_color([0, 0, 1])
        # line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
        # o3d.visualization.draw_geometries([pcd_new, source_temp, mesh_frame, ptc1, ptc2, line])

red
2
115.3049736499684
BOX EXTENT :  [79.16738022 37.0991582  17.82132286]
BOX CENTER COORDINATE :  [ -2.35121956  22.63981833 115.30497365]
PointCloud with 17914 points.
Initial alignment
RegistrationResult with fitness=6.023222e-01, inlier_rmse=7.350531e+00, and correspondence_set size of 10790
Access transformation to get result.
RegistrationResult with fitness=6.365971e-01, inlier_rmse=2.795055e+00, and correspondence_set size of 11404
Access transformation to get result.
Transformation is:
[[ 9.98513255e-01  5.01729087e-02  2.13063328e-02 -2.52144516e+00]
 [-5.33837021e-02  9.79122990e-01  1.96133504e-01  2.36347143e+01]
 [-1.10209319e-02 -1.96979315e-01  9.80345699e-01  1.09372191e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
NEW Transformation is:
[[ 9.98573900e-01  5.33869444e-02  0.00000000e+00 -2.52144516e+00]
 [-5.33869444e-02  9.98573900e-01  0.00000000e+00  2.36347143e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.09372191e+02]
 [ 0.0

In [ ]:
o3d.visualization.draw_geometries([source_temp, mesh_frame, source_temp.get_axis_aligned_bounding_box()])

In [ ]:
np.array(source_temp.get_axis_aligned_bounding_box().get_box_points())

array([[-42.53197249, -51.93269998,  -7.79647446],
       [ 51.06873855, -51.93269998,  -7.79647446],
       [-42.53197249,  55.06628926,  -7.79647446],
       [-42.53197249, -51.93269998, 184.10104697],
       [ 51.06873855,  55.06628926, 184.10104697],
       [-42.53197249,  55.06628926, 184.10104697],
       [ 51.06873855, -51.93269998, 184.10104697],
       [ 51.06873855,  55.06628926,  -7.79647446]])

In [ ]:
np.array(source_temp.points)[:,2].max()

184.10104696735988

In [ ]:
o3d.visualization.draw_geometries([pcd_new, mesh_frame])

In [ ]:
c = np.array([target_x, target_y, target_z, 1])

In [ ]:
vector

array([ 0, -1,  0])

In [ ]:
c #=np.append(vector, 1)

array([ -2.34921337, 122.63930743, 115.31527208,   1.        ])

In [ ]:
np.linalg.inv(trans)

array([[-8.61729636e-01,  5.06817205e-01, -2.36295412e-02,
        -1.44003445e+00],
       [ 1.90176583e-02, -1.42749296e-02, -9.99717238e-01,
         1.02189449e+02],
       [-5.07011206e-01, -8.61935350e-01,  2.66265347e-03,
         2.81797776e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
np.inner(np.linalg.inv(trans), c)

array([ 60.01521638, -14.88856036, 177.58874407,   1.        ])

In [ ]:
c2 = np.inner(np.linalg.inv(trans), c)[:3]

In [ ]:
c2

array([ 60.01521638, -14.88856036, 177.58874407])

In [ ]:
c3 = c2 - move
c3

array([ 38.7140904 , -21.44618395, 341.05220345])

In [ ]:
c4 = c3 / resize
c4

array([ 38.7140904 , -21.44618395, 341.05220345])

In [ ]:
cc = np.append(c4, 1)
cc

array([ 38.7140904 , -21.44618395, 341.05220345,   1.        ])

In [ ]:
c5 = np.inner(np.linalg.inv(np.asarray([[0, 0, -1, 0],
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, 0, 1]])), cc)[:3]

In [ ]:
c5

array([ -21.44618395, -341.05220345,  -38.7140904 ])

In [ ]:
resize

1

In [ ]:
extrinsic = np.array([[0.999957, 0.00923355, 0.0012111, -32.132],
                      [-0.00930628, 0.995592, 0.0933224, -1.93767],
                      [-0.00034407, -0.0933296, 0.995635, 3.9009],
                      [0,0,0,1]])

In [ ]:
ex_inv = np.linalg.inv(extrinsic)
ex_inv

array([[ 9.99956273e-01, -9.30627973e-03, -3.44064524e-04,
         3.21139046e+01],
       [ 9.23355132e-03,  9.95592893e-01, -9.33296850e-02,
         2.58989272e+00],
       [ 1.21110508e-03,  9.33224369e-02,  9.95635408e-01,
        -3.66413085e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
tar_x, tar_y, tar_z = tuple(c5)

In [ ]:
ptpt = o3d.cpu.pybind.utility.Vector3dVector(np.array([[tar_x, tar_y, tar_z], [tar_x - 0.00001, tar_y - 0.00001, tar_z - 0.00001], [tar_x + 0.00001, tar_y + 0.00001, tar_z + 0.00001]]))

In [ ]:
ptpt1 = o3d.geometry.PointCloud()
ptpt1.points = ptpt
ptpt1.paint_uniform_color([1, 0, 0])

PointCloud with 3 points.

In [ ]:
o3d.visualization.draw_geometries([pcd_combined, ptpt1])

In [ ]:
c6 = np.inner(ex_inv, np.append(c5,1))

In [ ]:
c6

array([  13.8559058 , -333.5441077 ,  -74.06304635,    1.        ])